Übung: kind
-----------

![](https://kind.sigs.k8s.io/logo/logo.png)

Quelle: https://kind.sigs.k8s.io/

- - -

Kind (Kubernetes IN Docker) ist ein leistungsstarkes Werkzeug, das Entwicklern und Administratoren hilft, Kubernetes-Cluster auf einfache und schnelle Weise lokal zu erstellen und zu verwalten. 

Kind wurde speziell entwickelt, um Kubernetes-Cluster in Docker-Containern zu betreiben, was es ideal für Entwicklungs-, Test- und CI/CD-Umgebungen macht.

* Einfache Installation und Nutzung: Da Kind Docker-Container verwendet, kann ein Kubernetes-Cluster mit nur wenigen Befehlen eingerichtet werden. Dies ermöglicht eine schnelle Bereitstellung und einen unkomplizierten Start.

* Portabilität: Durch die Nutzung von Docker-Containern sind die erstellten Kubernetes-Cluster plattformunabhängig und können auf verschiedenen Betriebssystemen wie Linux, macOS und Windows ausgeführt werden.

* Schnelle Iteration: Kind ermöglicht schnelle Iterationen bei der Entwicklung und beim Testen von Kubernetes-Anwendungen. Cluster können in Minuten erstellt und gelöscht werden, was eine flexible und dynamische Entwicklungsumgebung unterstützt.

* CI/CD-Integration: Kind eignet sich hervorragend für die Integration in Continuous Integration und Continuous Deployment (CI/CD)-Pipelines. Entwickler können ihre Anwendungen in einer realistischen Kubernetes-Umgebung testen, bevor sie in die Produktion überführt werden.

* Leichtgewichtig: Im Gegensatz zu herkömmlichen Kubernetes-Installationen benötigt Kind weniger Ressourcen, da es auf Docker-Containern basiert. Dies macht es zu einer idealen Wahl für Entwickler, die auf Laptops oder in begrenzten Entwicklungsumgebungen arbeiten.

- - -

In der Übung erstellen wir einen Kubernetes Cluster in einem Container. 

Dazu benötigen wir zuerst eine Konfigurationsdatei welche die Anzahl Master und Worker Nodes festlegt. Hier ein Master und drei Worker Nodes


In [ ]:
%%bash
cat <<%EOF% >kind-config.yaml
# kind-config.yaml
kind: Cluster
apiVersion: kind.x-k8s.io/v1alpha4
nodes:
  - role: control-plane
  - role: worker
  - role: worker
  - role: worker
%EOF%

Mit dieser Konfiguration können wir einen Kubernetes Cluster erstellen, wo jede Node als Container läuft.

In [ ]:
! kind create cluster --config kind-config.yaml --name docker-kind

Der Kubernetes Cluster muss als Docker Container sichtbar und via kubectl ansprechbar sein:

In [ ]:
! docker container ps
! kubectl get nodes

Intressant sind die gestartet Kubernetes Dienste, welche bei `kind` den Kubernetes Standards entsprechen

In [ ]:
! kubectl get pods -A -o wide

- - -

Zum Vergleich die Kubernetes Dienste von der [microk8s](https://microk8s.io/) Kubernetes Distribution.

Dazu brauchen wir zuerst die Context Namen unserer laufenden Kubernetes Clusters

In [ ]:
! kubectl config get-contexts

Mit diesem Context Namen können wir nun den Kubernetes Cluster wechseln

In [ ]:
%%bash
kubectl config use-context microk8s
kubectl get pods -A
echo "--------------------------------"
pgrep -l dqlite
pgrep -l kubelite
#pstree -npTA

Wir stellen fest, dass sich die Kubernetes Distributionen hauptsächlich 
* in der Art der verwendeteten Produkte (z.B. für Netzwerk kindnet vs. calico, Key-Value Server etcd vs. dqlite)
* ob die System Prozesse als Container oder Linux System Dienste laufen
  
unterscheiden.

- - -

Aufräumen

Dazu löschen wir den `kind` Kubernetes Cluster und wechseln auf unseren `microk8s` Kubernetes cluster

In [ ]:
! kind delete cluster --name docker-cluster